In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet201,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexByteplot"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet201(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1920)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1920)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.3min


[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.880079, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.895153, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.883929, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.889662, total= 3.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.877451, total= 3.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.885742, total= 3.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.882874, total= 3.4min
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  7.5min


[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.889216, total= 7.4min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.884540, total= 7.5min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.884273, total= 7.4min
[CV] ............................ C=0.1, score=0.876115, total= 7.4min
[CV] C=10.0 ..........................................................
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.868110, total= 7.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.891650, total= 7.4min
[CV] ............................ C=0.1, score=0.891602, total= 7.5min
[CV] C=100.0 .........................................................
[CV] C

...

[CV] ................... C=1000000000.0, score=0.879883, total= 6.4min
[CV] .................... C=100000000.0, score=0.866799, total= 6.5min
[CV] ................... C=1000000000.0, score=0.867647, total= 6.3min
[CV] ................... C=1000000000.0, score=0.871820, total= 6.4min
[CV] ................... C=1000000000.0, score=0.855315, total= 6.3min
[CV] ................... C=1000000000.0, score=0.879921, total= 6.3min
[CV] ................... C=1000000000.0, score=0.869522, total= 6.1min
[CV] ................... C=1000000000.0, score=0.874382, total= 6.3min
[CV] ................... C=1000000000.0, score=0.880716, total= 6.2min
[CV] .................. C=10000000000.0, score=0.874755, total= 4.1min
[CV] .................. C=10000000000.0, score=0.874510, total= 4.1min
[CV] .................. C=10000000000.0, score=0.879921, total= 4.0min
[CV] .................. C=10000000000.0, score=0.853175, total= 3.9min
[CV] .................. C=10000000000.0, score=0.866204, total= 4.0min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 37.3min remaining:  1.5min


[CV] .................. C=10000000000.0, score=0.868164, total= 4.5min
[CV] .................. C=10000000000.0, score=0.856299, total= 4.1min
[CV] .................. C=10000000000.0, score=0.867458, total= 4.0min
[CV] .................. C=10000000000.0, score=0.867530, total= 4.0min
[CV] .................. C=10000000000.0, score=0.867793, total= 4.1min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 37.5min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([200.20662682, 448.06471941, 444.22846682, 446.1227304 ,
        448.10692501, 448.14504902, 449.42547979, 447.71213319,
        446.9277082 , 447.84877262, 419.67508771, 372.68251884,
        244.4975565 ]),
 'mean_score_time': array([0.0658247 , 0.06358361, 0.05421927, 0.05264482, 0.05934246,
        0.05790367, 0.04827218, 0.05272777, 0.0478874 , 0.05900459,
        0.04077449, 0.01937892, 0.01377144]),
 'mean_test_score': array([0.88565509, 0.88447119, 0.87282952, 0.86878453, 0.86898185,
        0.86671271, 0.86779795, 0.86878453, 0.86908051, 0.87174428,
        0.86612076, 0.87046172, 0.86760063]),
 'mean_train_score': array([0.95151477, 0.98356802, 0.9889505 , 0.98668407, 0.9858028 ,
        0.98127444, 0.98816154, 0.98709791, 0.98675854, 0.98939923,
        0.98462167, 0.98789726, 0.98497421]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.88566, std: 0.00518, params: {'C': 0.01},
 mean: 0.88447, std: 0.00582, params: {'C': 0.1},
 mean: 0.87283, std: 0.00710, params: {'C': 1.0},
 mean: 0.86878, std: 0.00797, params: {'C': 10.0},
 mean: 0.86898, std: 0.00675, params: {'C': 100.0},
 mean: 0.86671, std: 0.01868, params: {'C': 1000.0},
 mean: 0.86780, std: 0.01042, params: {'C': 10000.0},
 mean: 0.86878, std: 0.00780, params: {'C': 100000.0},
 mean: 0.86908, std: 0.00823, params: {'C': 1000000.0},
 mean: 0.87174, std: 0.00994, params: {'C': 10000000.0},
 mean: 0.86612, std: 0.00906, params: {'C': 100000000.0},
 mean: 0.87046, std: 0.00821, params: {'C': 1000000000.0},
 mean: 0.86760, std: 0.00767, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.8857
